In [2]:
!conda install sunode -y

Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/homebrew/anaconda3/envs/ml-tutorials

  added / updated specs:
    - sunode


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    icu-73.2                   |       hc8870d7_0        11.4 MB  conda-forge
    libblas-3.9.0              |16_osxarm64_openblas          13 KB  conda-forge
    libcblas-3.9.0             |16_osxarm64_openblas          13 KB  conda-forge
    libhwloc-2.11.2            |default_h3f80f97_1000         2.2 MB  conda-forge
    liblapack-3.9.0            |16_osxarm64_openblas          13 KB  conda-forge
    libxml2-2.12.7             |       ha661575_1         575 KB  conda-forge
    llvmlite-0.42.0            |  py312h17030e7_1         362 KB  conda-forge
    numba-0.59.1               |  py312hbaff935_0         5.3 MB  conda-forge
    ----

In [4]:
def lotka_volterra(t, y, p):
    """Right hand side of Lotka-Volterra equation.

    All inputs are dataclasses of sympy variables, or in the case
    of non-scalar variables numpy arrays of sympy variables.
    """
    return {
        'hares': p.alpha * y.hares - p.beta * y.lynx * y.hares,
        'lynx': p.delta * y.hares * y.lynx - p.gamma * y.lynx,
    }


from sunode.symode import SympyProblem
problem = SympyProblem(
    params={
        # We need to specify the shape of each parameter.
        # Any empty tuple corresponds to a scalar value.
        'alpha': (),
        'beta': (),
        'gamma': (),
        'delta': (),
    },
    states={
        # The same for all state variables
        'hares': (),
        'lynx': (),
    },
    rhs_sympy=lotka_volterra,
    derivative_params=[
        # We need to specify with respect to which variables
        # gradients should be computed.
        ('alpha',),
        ('beta',),
    ],
)

# The solver generates uses numba and sympy to generate optimized C functions
# for the right-hand-side and if necessary for the jacobian, adoint and
# quadrature functions for gradients.
from sunode.solver import Solver
solver = Solver(problem, sens_mode=None, solver='BDF')


import numpy as np
tvals = np.linspace(0, 10)
# We can use numpy structured arrays as input, so that we don't need
# to think about how the different variables are stored in the array.
# This does not introduce any runtime overhead during solving.
y0 = np.zeros((), dtype=problem.state_dtype)
y0['hares'] = 1
y0['lynx'] = 0.1

# We can also specify the parameters by name:
solver.set_params_dict({
    'alpha': 0.1,
    'beta': 0.2,
    'gamma': 0.3,
    'delta': 0.4,
})

output = solver.make_output_buffers(tvals)
solver.solve(t0=0, tvals=tvals, y0=y0, y_out=output)

# We can convert the solution to an xarray Dataset
solver.as_xarray(tvals, output).solution_hares.plot()

# Or we can convert it to a numpy record array
from matplotlib import pyplot as plt
plt.plot(output.view(problem.state_dtype)['hares'])

ImportError: dlopen(/opt/homebrew/anaconda3/envs/ml-tutorials/lib/python3.12/site-packages/_sundials_cvodes.abi3.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib
  Referenced from: <3C8E0CA3-0686-305E-9FD2-504758DB420E> /opt/homebrew/anaconda3/envs/ml-tutorials/lib/libsundials_sunlinsollapackdense.3.8.0.dylib
  Reason: tried: '/opt/homebrew/anaconda3/envs/ml-tutorials/lib/liblapack.3.dylib' (no such file), '/opt/homebrew/anaconda3/envs/ml-tutorials/lib/python3.12/site-packages/../../liblapack.3.dylib' (no such file), '/opt/homebrew/anaconda3/envs/ml-tutorials/lib/python3.12/site-packages/../../liblapack.3.dylib' (no such file), '/opt/homebrew/anaconda3/envs/ml-tutorials/bin/../lib/liblapack.3.dylib' (no such file), '/opt/homebrew/anaconda3/envs/ml-tutorials/bin/../lib/liblapack.3.dylib' (no such file), '/usr/local/lib/liblapack.3.dylib' (no such file), '/usr/lib/liblapack.3.dylib' (no such file, not in dyld cache)

In [ ]:
import jax.numpy as jnp
from sunode import 
import sunode.sympy as sp



ImportError: dlopen(/opt/homebrew/anaconda3/envs/ml-tutorials/lib/python3.12/site-packages/_sundials_cvodes.abi3.so, 0x0002): Library not loaded: @rpath/liblapack.3.dylib
  Referenced from: <3C8E0CA3-0686-305E-9FD2-504758DB420E> /opt/homebrew/anaconda3/envs/ml-tutorials/lib/libsundials_sunlinsollapackdense.3.8.0.dylib
  Reason: tried: '/opt/homebrew/anaconda3/envs/ml-tutorials/lib/liblapack.3.dylib' (no such file), '/opt/homebrew/anaconda3/envs/ml-tutorials/lib/python3.12/site-packages/../../liblapack.3.dylib' (no such file), '/opt/homebrew/anaconda3/envs/ml-tutorials/lib/python3.12/site-packages/../../liblapack.3.dylib' (no such file), '/opt/homebrew/anaconda3/envs/ml-tutorials/bin/../lib/liblapack.3.dylib' (no such file), '/opt/homebrew/anaconda3/envs/ml-tutorials/bin/../lib/liblapack.3.dylib' (no such file), '/usr/local/lib/liblapack.3.dylib' (no such file), '/usr/lib/liblapack.3.dylib' (no such file, not in dyld cache)

In [ ]:

def model(t, y, p):
    # State variables (example naming):
    # M(t): market size at time t
    # C(t): number of customers at time t
    # Q(t): product quality (maturity level) at time t
    # B(t): budget at time t
    #
    # Decision/control parameters (could be made time-varying or constants):
    # α(t): fraction of engineering resources allocated to product improvement
    # 1 - α(t): fraction of engineering resources allocated to supporting customers
    #
    # Additional parameters in p:
    # p['E']: total engineering resources
    # p['S']: total sales/outreach resources
    # p['λ']: propensity of customers to leave if not enough value delivered
    # p['γ']: rate at which we acquire customers from the market
    # p['μ']: rate of market shrinkage
    # p['c_dev']: cost per unit engineering effort
    # p['c_sales']: cost per unit sales effort (if any)
    #
    # Value per customer per unit time depends on Q:
    # If Q is large, less engineering resource is needed per customer.
    # Engineering needed per customer ~ 1/Q (assuming Q>0)
    #
    # Product improvement dynamics:
    # dQ/dt ~ α * E * C
    # (assuming product improvement speed is influenced by having customers: they guide development)
    #
    # Customer support:
    # Customers gain value from (1-α)*E units of engineering spread over C customers:
    # Value delivered per customer ~ ((1-α)*E) / C * Q
    # (Scaling by Q: with better quality, each unit of engineering support yields more effective value)
    #
    # Probability of a customer leaving ~ λ * max(0, (V_threshold - delivered_value))
    # For simplicity, let’s say V_threshold = 1 unit of value/time.
    # If delivered_value < 1, fraction leaving per unit time ~ λ*(1 - delivered_value)
    # dC/dt = (new customers) - (customers leaving)
    # new customers ~ γ * S * (M - C) (from remaining market)
    # leaving ~ λ*(1 - delivered_value)*C if delivered_value < 1 else 0
    #
    # Market dynamics:
    # dM/dt = -μ*M
    #
    # Budget dynamics:
    # Revenue ~ C * R (some revenue per customer, say R per customer per unit time)
    # Costs ~ c_dev * E + c_sales * S
    # dB/dt = C*R - (c_dev*E + c_sales*S)
    #
    # This is a rough template; you can adjust functional forms as needed.

    M, C, Q, B = y

    # Extract parameters
    E = p['E']
    S = p['S']
    lam = p['lambda']  # λ
    gamma = p['gamma'] # γ
    mu = p['mu']       # μ
    c_dev = p['c_dev']
    c_sales = p['c_sales']
    R = p['R']         # revenue per customer
    alpha = p['alpha'] # fraction of eng. on product

    # Value delivered per customer
    # If C == 0, handle division by zero: no customers means no delivered_value needed
    delivered_value = jnp.where(C > 0, ((1 - alpha)*E / C) * Q, 0.0)

    # Customer attrition
    leave_rate = jnp.where(delivered_value < 1.0, lam*(1.0 - delivered_value), 0.0)
    dCdt = gamma * S * (M - C) - leave_rate * C

    # Product improvement
    dQdt = alpha * E * C

    # Market shrinkage
    dMdt = -mu * M

    # Budget change
    dBdt = C*R - (c_dev*E + c_sales*S)

    return jnp.array([dMdt, dCdt, dQdt, dBdt])


In [ ]:


# Example usage:
# Define parameters
params = {
    'E': 10.0,       # total engineering resources
    'S': 5.0,        # total sales resources
    'lambda': 0.5,   # leaving propensity
    'gamma': 0.1,    # acquisition rate
    'mu': 0.01,      # market shrink rate
    'c_dev': 100.0,  # cost per eng. unit
    'c_sales': 50.0, # cost per sales unit
    'R': 500.0,      # revenue per customer
    'alpha': 0.5     # fraction of engineering for product
}


In [ ]:


# Initial conditions: M0, C0, Q0, B0
y0 = jnp.array([10000.0, 10.0, 1.0, 100000.0])


In [ ]:

res = diffeqsolve(
    rhs=model,
    t0=0.0,
    t1=100.0,
    y0=y0,
    params=params,
    solver='RK45',
    dt0=0.1
)

# res.ys will contain the solution arrays for M(t), C(t), Q(t), B(t)